In [1]:
"""
Setup:
pip3 install elasticsearch
deploy chart-repo/eric-data-search-engine, chart-repo/eric-data-search-engine, local BRO build
port forwarding eric-data-search-engine-ingest-*:9200 to  localhost:9200
port forwarding eric-ctrl-bro-0:7001 to localhost:7001
"""

'\nSetup:\npip3 install elasticsearch\ndeploy chart-repo/eric-data-search-engine, chart-repo/eric-data-search-engine, local BRO build\nport forwarding eric-data-search-engine-ingest-*:9200 to  localhost:9200\nport forwarding eric-ctrl-bro-0:7001 to localhost:7001\n'

In [22]:
from elasticsearch import Elasticsearch
es = Elasticsearch('https://10.120.217.249:9200', verify_certs=False)

/home/ubuntu/.local/lib/python3.8/site-packages/elasticsearch/connection/http_urllib3.py:204: UserWarning: Connecting to https://10.120.217.249:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [7]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [20]:
import json

import requests as r

In [23]:
es.info()

/home/ubuntu/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
/home/ubuntu/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
/home/ubuntu/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
/home/ubuntu/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS r

SSLError: ConnectionError([SSL: SSLV3_ALERT_BAD_CERTIFICATE] sslv3 alert bad certificate (_ssl.c:2635)) caused by: SSLError([SSL: SSLV3_ALERT_BAD_CERTIFICATE] sslv3 alert bad certificate (_ssl.c:2635))

In [11]:
es.cat.indices()

'green open adp-app-logs-2021.08.31 dbciACtQQk-hwp2wZ_P-jA 5 1  11 0 213.1kb 106.4kb\ngreen open adp-app-logs-2021.08.30 liWAfXR8RxuoZ9W3hhy-UQ 5 1 388 0 851.8kb 454.4kb\ngreen open adp-app-logs-2021.08.27 W__DEx4XTUGs_vM2eu1IPg 5 1  72 0 600.5kb 307.2kb\ngreen open adp-app-logs-2021.08.26 TCr5eB7PS_adm597pbCmAw 5 1  91 0 734.6kb 369.2kb\ngreen open adp-app-logs-2021.08.25 PHOB-anOS8CVlnk-KoqD5A 5 1  10 0 213.9kb 106.9kb\ngreen open adp-app-logs-2021.08.24 -jM_cvzlQXCHt2k8CA75kQ 5 1  53 0 579.4kb 289.7kb\ngreen open my-index-01             2mnlnLP0RquSvwWrcq4VFw 5 1   0 0     2kb     1kb\n'

In [12]:
index = es.cat.indices().split("\n")[0].split(" ")[2]
index

'adp-app-logs-2021.08.31'

In [16]:
"check the logs so far"
res = es.search(
    index=index,
    body={"query": {"match": {"service_id" : "eric-ctrl-bro"}}},
    sort = "@timestamp:ascending",
    size=10000)
res = sorted(res["hits"]["hits"], key=lambda x: x["_source"]["@timestamp"])
#data = [(x["_source"]["@timestamp"] +" : " + x["_source"]["severity"] + " : " + x["_source"]["message"]) for x in res]

print(json.dumps(res, indent=2))

[
  {
    "_index": "adp-app-logs-2021.08.31",
    "_type": "_doc",
    "_id": "SZyNnHsBJeAju-avjVh5",
    "_score": 0.15387988,
    "_source": {
      "port": 2399,
      "tags": [
        "syslog5424"
      ],
      "facility": "local0",
      "service_id": "eric-ctrl-bro",
      "type": "syslog",
      "metadata": {
        "proc_id": "19868"
      },
      "logplane": "adp-app-logs",
      "version": "1.0.0",
      "timestamp": "2021-08-31T14:12:06.619Z",
      "kubernetes": {
        "node": {
          "name": "IE-00002941"
        }
      },
      "severity": "info",
      "message": "Starting BackupAndRestoreApplication using Java 14.0.2 on IE-00002941 with PID 19868 (C:\\Users\\eyicwux\\Documents\\Ericsson\\PlanB\\BRO\\64784\\backupandrestore\\service\\target\\classes started by eyicwux in C:\\Users\\eyicwux\\Documents\\Ericsson\\PlanB\\BRO\\64784\\backupandrestore\\service)",
      "@timestamp": "2021-08-31T14:12:06.815Z",
      "@version": "1"
    }
  },
  {
    "_index": "a

In [17]:
len(res)

46

In [53]:
# cause BRO to spit out an error
body = {
    "action": "CREATE_BACKUP",
    "payload": {
#        "backupName": "myBackup"
        "backupName" : dumb_name_250
    }
}
res = r.post("http://localhost:7001/v1/backup-manager/DEFAULT/action", json=body)

In [2]:
import requests
baseurl = 'http://10.120.217.249:9200'

In [4]:
result = requests.get(baseurl)
result.json()

{'name': 'eric-data-search-engine-ingest-895bbccf7-7ddnm',
 'cluster_name': 'eric-data-search-engine',
 'cluster_uuid': 'YbusCkk7SxGN2Fp_xdW4Kw',
 'version': {'number': '7.8.1',
  'build_flavor': 'oss',
  'build_type': 'tar',
  'build_hash': 'b5ca9c58fb664ca8bf9e4057fc229b3396bf3a89',
  'build_date': '2020-07-21T16:40:44.668009Z',
  'build_snapshot': False,
  'lucene_version': '8.5.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
url = baseurl + '/_search'
body={"query": {"match_all": {}}},
result = requests.post(url, json = body)
result.text

'{"error":{"root_cause":[{"type":"parsing_exception","reason":"Expected [START_OBJECT] but found [START_ARRAY]","line":1,"col":1}],"type":"parsing_exception","reason":"Expected [START_OBJECT] but found [START_ARRAY]","line":1,"col":1},"status":400}'